# Convolutional Neural Network

### Part 1 - Building the CNN

#### Importing the Tensorflow libraries and packages

In [1]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.1.0
Keras Version: 2.2.4-tf

Python 3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.4
Scikit-Learn 0.24.1
GPU is available


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau

import os

In [3]:
TRAIN_DIR = 'dataset/train'
TEST_DIR = 'dataset/test'
VALID_DIR = 'dataset/valid'
IMG_SIZE = 224
IMG_RESCALE = 255

In [4]:
classifier = Sequential()

#classes = 250

#Kernels usually 1x1 | 3x3 | 5x5

#Adding a convolutional layer
classifier.add(Conv2D(32, (3, 3), input_shape = (IMG_SIZE, IMG_SIZE, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a second convolutional layer
classifier.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a third convolutional layer
classifier.add(Conv2D(128, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a fourth convolutional layer
classifier.add(Conv2D(256, (3, 3), input_shape = (IMG_SIZE, IMG_SIZE, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Flattening
classifier.add(Flatten())

#Full connection
classifier.add(Dense(units = 1024, activation = 'relu'))
classifier.add(Dropout(0.10))
classifier.add(Dense(units = 250, activation= 'softmax'))

Let's double check the model description:

In [5]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 256)       2

In [6]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Part 2 - Fitting the CNN to the images


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rescale = 1./IMG_RESCALE,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./IMG_RESCALE)

validation_datagen = ImageDataGenerator(rescale = 1./IMG_RESCALE)

In [8]:
training_set = train_datagen.flow_from_directory(TRAIN_DIR,
                                                 target_size = (IMG_SIZE, IMG_SIZE),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(TEST_DIR,
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            batch_size = 32,
                                            class_mode = 'categorical')

validation_set = validation_datagen.flow_from_directory(VALID_DIR,
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 35215 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.


In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime

#rm -rf /logs/

es = EarlyStopping(monitor='val_loss', patience=6)
mc = ModelCheckpoint('best_model_3.h5', monitor='val_loss', mode='min', save_best_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4)

# Create a TensorBoard callback
log_dir = "logs\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
classifier.fit( training_set,
                         epochs = 25,
                         callbacks = [tensorboard_callback,es,mc,reduceLR],
                         validation_data = validation_set,
                         workers = 4)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1101 steps, validate for 40 steps
Epoch 1/25
1101/1101 [==============================] - 105s 96ms/step - loss: 5.1790 - accuracy: 0.0363 - val_loss: 4.0936 - val_accuracy: 0.1696
Epoch 2/25
1101/1101 [==============================] - 104s 94ms/step - loss: 3.4383 - accuracy: 0.2617 - val_loss: 2.3419 - val_accuracy: 0.4656
Epoch 3/25
1101/1101 [==============================] - 104s 94ms/step - loss: 2.4003 - accuracy: 0.4479 - val_loss: 1.7244 - val_accuracy: 0.5976
Epoch 4/25
1101/1101 [==============================] - 104s 95ms/step - loss: 1.8635 - accuracy: 0.5543 - val_loss: 1.4326 - val_accuracy: 0.6488
Epoch 5/25
1101/1101 [==============================] - 105s 95ms/step - loss: 1.5366 - accuracy: 0.6226 - val_loss: 1.2554 - val_accuracy: 0.6920
Epoch 6/25
1101/1101 [==============================] - 104s 95ms/step - loss: 1.3004 - accuracy: 0.6703 - val_loss: 1.2410 - val_accuracy: 0.7096
Epoch 7/25
1101/1101 [==

In [11]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 6544), started 5:22:14 ago. (Use '!kill 6544' to kill it.)

# Part 3 test the CNN

In [12]:
# Load Model
# modelo 3
classifier.load_weights('best_model_3.h5')

In [13]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 256)       2

In [14]:
score = classifier.evaluate(test_set, verbose=0)

print('Test Loss', score[0])
print('Test accuracy', score[1])

  ...
    to  
  ['...']
Test Loss 0.9466227665543556
Test accuracy 0.7888
